In [1]:
! pip install optbinning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.2/211.2 kB 860.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.2/296.2 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.2 MB/s eta 0:00:00


In [2]:
import joblib
import numpy as np
import pandas as pd
import gc
import time
import os
import sys
from contextlib import contextmanager
from lightgbm import LGBMClassifier
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, roc_curve, f1_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from bayes_opt import BayesianOptimization
from skopt import BayesSearchCV 
from optbinning import OptimalBinning
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from yellowbrick.cluster import KElbowVisualizer
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from sklearn.decomposition import PCA
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
import random
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
pd.set_option('display.max_rows', 500)

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(1)

In [5]:
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances01.png')

In [6]:
# Plots the disribution of a variable colored by value of the target
def kde_target(var_name, df):
    
    # Calculate the correlation coefficient between the new variable and the target
    corr = df['Y_LABEL'].corr(df[var_name])
    
    # Calculate medians for repaid vs not repaid
    avg_target_0 = df.loc[df['Y_LABEL'] == 0, var_name].median()
    avg_target_1 = df.loc[df['Y_LABEL'] == 1, var_name].median()
    
    plt.figure(figsize = (12, 6))
    
    # Plot the distribution for target == 0 and target == 1
    sns.kdeplot(df.loc[df['Y_LABEL'] == 0, var_name], label = 'Y_LABEL == 0')
    sns.kdeplot(df.loc[df['Y_LABEL'] == 1, var_name], label = 'Y_LABEL == 1')
    
    # label the plot
    plt.xlabel(var_name); plt.ylabel('Density'); plt.title('%s Distribution' % var_name)
    plt.legend();
    
    # print out the correlation
    print('The correlation between %s and the TARGET is %0.4f' % (var_name, corr))
    # Print out average values
    print('Median value for avg_target_0 = %0.4f' % avg_target_0)
    print('Median value for avg_target_1 = %0.4f' % avg_target_1)

In [7]:
def train_model(train,test,params,stratified,num_folds,drop_features,seed_num=1):
    
    seed_everything(seed_num)
    
    # Divide in training/validation and test data
    train_df = train.copy()
    test_df = test.copy()

    # label encoding 
    encoder = LabelEncoder()
    categorical_features = [i for i in train_df.select_dtypes(include=['object','category']).columns.tolist() if i not in ['ID']]
    categorical_features = [i for i in categorical_features if i in train_df.columns.tolist()]
    for each in categorical_features:
        train_df[each] = encoder.fit_transform(train_df[each])
        test_df[each] = encoder.fit_transform(test_df[each])

    # set training options
    stratified = stratified
    num_folds = num_folds

    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed_num)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=seed_num)

    # Create arrays and dataframes to store results
    oof_preds_lgb = np.zeros(train_df.shape[0])
    sub_preds_lgb = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['Y_LABEL','ID','SAMPLE_TRANSFER_DAY']+drop_features]

    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['Y_LABEL'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['Y_LABEL'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['Y_LABEL'].iloc[valid_idx]

        """        
        learning_rate=0.02,
        num_leaves=34,
        colsample_bytree=0.2,
        subsample=0.8715623,
        max_depth=4,
        reg_alpha=0.041545473,
        reg_lambda=0.0735294,
        min_split_gain=0.0222415,
        min_child_weight=39.3259775,
        min_child_samples = 10,
        """

        # LightGBM parameters found by Bayesian optimization
        clf = LGBMClassifier(
            
            learning_rate = params['learning_rate'],
            num_leaves = int(round(params['num_leaves'])),
            colsample_bytree = params['colsample_bytree'],
            subsample = params['subsample'],
            max_depth = int(round(params['max_depth'])),
            reg_alpha = params['reg_alpha'],
            reg_lambda = params['reg_lambda'],
            min_split_gain = params['min_split_gain'],
            min_child_weight = params['min_child_weight'],
            min_child_samples = int(round(params['min_child_samples'])),    
            
            n_jobs = -1,
            n_estimators = 10000,            
            random_state = seed_num,
            silent=-1,
            deterministic=True,
            verbose=-1
        )
        
        with warnings.catch_warnings():
            
            warnings.filterwarnings('ignore')

            clf.fit(
                  train_x
                , train_y
                , eval_set=[(train_x, train_y), (valid_x, valid_y)]
                , eval_metric= 'auc'
                , verbose= 200
                , early_stopping_rounds= 500
            )

        oof_preds_lgb[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
        sub_preds_lgb += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds_lgb[valid_idx])))

    print('Full AUC score %.6f' % roc_auc_score(train_df['Y_LABEL'], oof_preds_lgb))

    # Write submission file and plot feature importance
    test_df['Y_LABEL_lgb'] = sub_preds_lgb

    # vi
    print('-'*50)
    # display(feature_importance_df.groupby(['feature'])['importance'].sum().sort_values(ascending=False).head(20))
    # print('-'*50)
    # display_importances(feature_importance_df)

    # find the best thred for f1-score
    f1_score_df = pd.DataFrame()
    for thred in [i/1000 for i in range(0,1000,1)]:

        a1 = pd.DataFrame()
        f1 = f1_score(train_df['Y_LABEL'], np.where(oof_preds_lgb>thred,1,0), average='macro')
        a1['f1'] = [f1]
        a1['thred'] = [thred]
        f1_score_df = pd.concat([f1_score_df, a1], axis=0)

    thred = f1_score_df.loc[f1_score_df['f1']==f1_score_df['f1'].max(),'thred'].tolist()[0]
    print('thred:',thred)
    print('ncol',len(feats))

    # train err
    a1 = roc_auc_score(train_df['Y_LABEL'], oof_preds_lgb)
    print('auc:',a1)
    f1 = f1_score(train_df['Y_LABEL'], np.where(oof_preds_lgb>thred,1,0), average='macro')
    print('f1:',f1)
    a1 = train_df['Y_LABEL'].value_counts()/len(train_df)
    print('Target ratio(real):',(a1[1]))

    # test err
    test_df['TARGET'] = np.where(test_df['Y_LABEL_lgb']>thred,1,0)
    a1 = test_df['TARGET'].value_counts()/len(test_df)
    print('Target ratio(pred):',(a1[1]))
    print('Target sum:',test_df['TARGET'].sum())

    # save 
    train_df['Y_LABEL_lgb'] = oof_preds_lgb
    a1 = train_df[['ID','YEAR','COMPONENT_ARBITRARY','Y_LABEL_lgb','Y_LABEL']].copy()
    a1.to_csv('train_pred_'+str(seed_num)+'_'+str(np.round(f1,10))+'.csv', index= False)    
    a1 = test_df[['ID','YEAR','COMPONENT_ARBITRARY','Y_LABEL_lgb']].copy()
    a1.to_csv('test_pred_'+str(seed_num)+'_'+str(np.round(f1,10))+'.csv', index= False)
    
    # submit
    a1 = test_df[['ID', 'TARGET']].copy()
    a1 = a1.rename(columns={'TARGET':'Y_LABEL'})
    submission_file_name = 'sample_submission_lgb_'+str(np.round(f1,4))+'.csv'
    a1.to_csv(submission_file_name, index= False)

In [8]:
def bayes_parameter_opt_lgb(
    train, 
    opt_params, 
    init_round=15, 
    opt_round=25, 
    n_folds=3, 
    random_seed=1, 
    n_estimators=10000, 
    output_process=False, 
    drop_features=[]
    ):   
    
    seed_everything(1)
    
    train_df = train.copy()

    # label encoding 
    encoder = LabelEncoder()
    categorical_features = [i for i in train_df.select_dtypes(include=['object','category']).columns.tolist() if i not in ['ID']]
    categorical_features = [i for i in categorical_features if i in train_df.columns.tolist()]
    for each in categorical_features:
        train_df[each] = encoder.fit_transform(train_df[each])
        
    # feats = [f for f in train_df.columns if f not in ['Y_LABEL','ID','SAMPLE_TRANSFER_DAY']]    
    # X = train_df[feats].copy()
    # y = train_df['Y_LABEL'].copy()
    
    # prepare data
    # train_data = lgb.Dataset(data=X, label=y, free_raw_data=False)
    
    # parameters
    def lgb_eval(
          learning_rate
        , num_leaves
        , colsample_bytree
        , subsample
        , max_depth
        , reg_alpha
        , reg_lambda
        , min_split_gain
        , min_child_weight
        , min_child_samples

    ):
        
        params = {'application':'binary', 'metric':'auc'}
        params['learning_rate'] = max(min(learning_rate, 1), 0)
        params["num_leaves"] = int(round(num_leaves))
        params['colsample_bytree'] = max(min(colsample_bytree, 1), 0)
        params['subsample'] = max(min(subsample, 1), 0)
        params['max_depth'] = int(round(max_depth))        
        params['reg_alpha'] = reg_alpha
        params['reg_lambda'] = reg_lambda        
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        params['min_child_samples'] = int(round(min_child_samples))
        
        # cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, verbose_eval =200, metrics=['auc'])
        
        # -----        
        
        stratified = True
        
        # Cross validation model
        if stratified:
            folds = StratifiedKFold(n_splits= 5, shuffle=True, random_state=1)
        else:
            folds = KFold(n_splits= 5, shuffle=True, random_state=1)

        # Create arrays and dataframes to store results
        oof_preds_lgb = np.zeros(train_df.shape[0])

        feats = [f for f in train_df.columns if f not in ['Y_LABEL','ID','SAMPLE_TRANSFER_DAY']+drop_features]

        for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['Y_LABEL'])):
            train_x, train_y = train_df[feats].iloc[train_idx], train_df['Y_LABEL'].iloc[train_idx]
            valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['Y_LABEL'].iloc[valid_idx]

            # LightGBM parameters found by Bayesian optimization
            clf = LGBMClassifier(

                **params,

                n_jobs = -1,
                n_estimators = 10000,            
                random_state = 1,
                silent=True,
                deterministic=True,
                verbose=-100
            )

            with warnings.catch_warnings():

                warnings.filterwarnings('ignore')

                clf.fit(
                      train_x
                    , train_y
                    , eval_set=[(train_x, train_y), (valid_x, valid_y)]
                    , eval_metric= 'auc'
                    , verbose= False
                    , early_stopping_rounds= 500
                )

            oof_preds_lgb[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]

        cv_result = roc_auc_score(train_df['Y_LABEL'], oof_preds_lgb)        
       
        # ----

        return cv_result

    lgbBO = BayesianOptimization(lgb_eval, opt_params, random_state=1)
    
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore')
        lgbBO.maximize(init_points=init_round, n_iter=opt_round, acq='ucb')
    
    model_auc=[]
    for model in range(len( lgbBO.res)):
        model_auc.append(lgbBO.res[model]['target'])
    
    a1 = lgbBO.res[pd.Series(model_auc).idxmax()]['target'],lgbBO.res[pd.Series(model_auc).idxmax()]['params']
    file_name = 'res_tune_'+str(len(drop_features))+'_'+str(a1[0])+'.joblib'    
    joblib.dump(a1[1],file_name)
    
    return a1

In [9]:
# read data
train = pd.read_csv('/kaggle/input/dacon001/open/train.csv')
test = pd.read_csv('/kaggle/input/dacon001/open/test.csv')
sample_submission = pd.read_csv('/kaggle/input/dacon001/open/sample_submission.csv')
original_columns = test.columns.tolist()

In [10]:
# test데이터셋에 있는 변수만 선택 
train = train[test.columns.to_list()+['Y_LABEL','SAMPLE_TRANSFER_DAY']].copy()

## // 파생변수 생성

In [11]:
"""
new features 
"""

train['YEAR_COMPONENT'] = train[['YEAR','COMPONENT_ARBITRARY']].apply(lambda x: '-'.join(x.astype(str)),axis=1)
test['YEAR_COMPONENT'] = test[['YEAR','COMPONENT_ARBITRARY']].apply(lambda x: '-'.join(x.astype(str)),axis=1)

var = [i for i in test.columns.tolist() if i not in ['ID','COMPONENT_ARBITRARY']]
train['zero_cnt'] = train[var].apply(lambda x:(x==0).sum(),axis=1)
test['zero_cnt'] = test[var].apply(lambda x:(x==0).sum(),axis=1)

"""
err features
"""

# train['ANONYMOUS_1_2000'] = np.where(train['ANONYMOUS_1']>2000,1,0)
# train['ANONYMOUS_2_200'] = np.where(train['ANONYMOUS_2']>200,1,0)
# train['CO_1.5'] = np.where(train['CO']>1.5,1,0)
# train['CR_100'] = np.where(train['CR']>100,1,0)
# train['FE_5000'] = np.where(train['FE']>5000,1,0)
# train['MN_5000'] = np.where(train['MN']>55,1,0)
# train['NI_14'] = np.where(train['NI']>14,1,0)

# test['ANONYMOUS_1_2000'] = np.where(test['ANONYMOUS_1']>2000,1,0)
# test['ANONYMOUS_2_200'] = np.where(test['ANONYMOUS_2']>200,1,0)
# test['CO_1.5'] = np.where(test['CO']>1.5,1,0)
# test['CR_100'] = np.where(test['CR']>100,1,0)
# test['FE_5000'] = np.where(test['FE']>5000,1,0)
# test['MN_5000'] = np.where(test['MN']>55,1,0)
# test['NI_14'] = np.where(test['NI']>14,1,0)

'\nerr features\n'

In [12]:
"""
transformation 
"""

for var in ['ANONYMOUS_1','PQINDEX','V40','ZN','FE']:
    
    train[var+'_log1'] = np.log(train[var]+1)
    test[var+'_log1'] = np.log(test[var]+1)

In [13]:
"""
normalization
""" 

for var in ['ZN','FE','CU','V40','MO','CR','NI','ANONYMOUS_1','ANONYMOUS_2']:
    
    ZN_min = train[var].min()
    ZN_max = train[var].max()
    train[var] = (train[var] - ZN_min) / (ZN_max-ZN_min)
    test[var] = (test[var] - ZN_min) / (ZN_max-ZN_min)

In [14]:
"""
feature combination
"""

seed_everything(1)

from itertools import chain, product 
candidate_var = ['ZN','FE','CU','V40','MO','CR','NI','ANONYMOUS_1','ANONYMOUS_2']
pairs = list(chain(product(candidate_var, candidate_var), product(candidate_var, candidate_var))) 
pairs = pd.Series([sorted([i,j]) for (i,j) in set(pairs) if i!=j]).drop_duplicates().reset_index(drop=True).tolist()
pairs = sorted(pairs)
# pairs = pairs.sort_values()
# random.shuffle(pairs)
# print(pairs)

for i in range(len(pairs)):

    train[pairs[i][0]+'_'+pairs[i][1]] = train[pairs[i][0]] * train[pairs[i][1]]
    test[pairs[i][0]+'_'+pairs[i][1]] = test[pairs[i][0]] * test[pairs[i][1]] 

In [15]:
"""
woe features 
"""

for variable in ['ANONYMOUS_1','PQINDEX','V40','ZN','FE']+['ANONYMOUS_1_V40', 'ANONYMOUS_1_CR', 'ANONYMOUS_1_ZN','ANONYMOUS_1_FE']:

    optb = OptimalBinning(name=variable, dtype="numerical", solver="cp")

    x = train[variable].values
    y = train['Y_LABEL']

    optb.fit(x, y)

    binning_table = optb.binning_table

    a1 = binning_table.build()
    
    # check 
    # print('iv:',a1['IV'].sum())
    # display(a1)
    # binning_table.plot(metric="event_rate")

    train[variable+'_woe_bin'] = pd.cut(train[variable],bins=optb.splits.tolist()).astype(str)
    test[variable+'_woe_bin'] = pd.cut(test[variable],bins=optb.splits.tolist()).astype(str)

In [16]:
"""
group operations
"""

opt_numeric_var = ['ANONYMOUS_1_V40', 'ANONYMOUS_1_CR', 'ANONYMOUS_1_ZN','ANONYMOUS_1_FE']

for group in ['COMPONENT_ARBITRARY','YEAR_COMPONENT','YEAR']+['FE_woe_bin']:

    for numeric_var in ['ZN','FE','V40','PQINDEX','CU','ANONYMOUS_1','ANONYMOUS_2']+opt_numeric_var:
        
        train = train.copy()
        test = test.copy()

        a1 = train.groupby([group])[numeric_var].median().to_dict()

        train[numeric_var+'_'+group+'_'+'median'] = train[group].map(a1)
        test[numeric_var+'_'+group+'_'+'median'] = test[group].map(a1)

        train[numeric_var+'_minus_'+numeric_var+'_'+group+'_'+'median'] = train[numeric_var] - train[numeric_var+'_'+group+'_'+'median']
        test[numeric_var+'_minus_'+numeric_var+'_'+group+'_'+'median'] = test[numeric_var] - test[numeric_var+'_'+group+'_'+'median']
        
        train[numeric_var+'_divide_'+numeric_var+'_'+group+'_'+'median'] = np.log(train[numeric_var]+0.00001) / np.log(train[numeric_var+'_'+group+'_'+'median']+0.00001) 
        test[numeric_var+'_divide_'+numeric_var+'_'+group+'_'+'median'] = np.log(test[numeric_var]+0.00001) / np.log(test[numeric_var+'_'+group+'_'+'median']+0.00001)
        
        
        a1 = train.groupby([group])[numeric_var].max().to_dict()

        train[numeric_var+'_'+group+'_'+'max'] = train[group].map(a1)
        test[numeric_var+'_'+group+'_'+'max'] = test[group].map(a1)

        train[numeric_var+'_minus_'+numeric_var+'_'+group+'_'+'max'] = train[numeric_var] - train[numeric_var+'_'+group+'_'+'max']
        test[numeric_var+'_minus_'+numeric_var+'_'+group+'_'+'max'] = test[numeric_var] - test[numeric_var+'_'+group+'_'+'max']
        
        train[numeric_var+'_divide_'+numeric_var+'_'+group+'_'+'max'] = np.log(train[numeric_var]+0.00001) / np.log(train[numeric_var+'_'+group+'_'+'max']+0.00001)
        test[numeric_var+'_divide_'+numeric_var+'_'+group+'_'+'max'] = np.log(test[numeric_var]+0.00001) / np.log(test[numeric_var+'_'+group+'_'+'max']+0.00001)
        
        
        a1 = train.groupby([group])[numeric_var].min().to_dict()

        train[numeric_var+'_'+group+'_'+'min'] = train[group].map(a1)
        test[numeric_var+'_'+group+'_'+'min'] = test[group].map(a1)

        train[numeric_var+'_minus_'+numeric_var+'_'+group+'_'+'min'] = train[numeric_var] - train[numeric_var+'_'+group+'_'+'min']
        test[numeric_var+'_minus_'+numeric_var+'_'+group+'_'+'min'] = test[numeric_var] - test[numeric_var+'_'+group+'_'+'min']
        
        train[numeric_var+'_divide_'+numeric_var+'_'+group+'_'+'min'] = np.log(train[numeric_var]+0.00001) / np.log(train[numeric_var+'_'+group+'_'+'min']+0.00001) 
        test[numeric_var+'_divide_'+numeric_var+'_'+group+'_'+'min'] = np.log(test[numeric_var]+0.00001) / np.log(test[numeric_var+'_'+group+'_'+'min']+0.00001) 
        
        
        a1 = train.groupby([group])[numeric_var].sum().to_dict()

        train[numeric_var+'_'+group+'_'+'sum'] = train[group].map(a1)
        test[numeric_var+'_'+group+'_'+'sum'] = test[group].map(a1)

        train[numeric_var+'_minus_'+numeric_var+'_'+group+'_'+'sum'] = train[numeric_var] - train[numeric_var+'_'+group+'_'+'sum']
        test[numeric_var+'_minus_'+numeric_var+'_'+group+'_'+'sum'] = test[numeric_var] - test[numeric_var+'_'+group+'_'+'sum']
        
        train[numeric_var+'_divide_'+numeric_var+'_'+group+'_'+'sum'] = np.log(train[numeric_var]+0.00001) / np.log(train[numeric_var+'_'+group+'_'+'sum']+0.00001)
        test[numeric_var+'_divide_'+numeric_var+'_'+group+'_'+'sum'] = np.log(test[numeric_var]+0.00001) / np.log(test[numeric_var+'_'+group+'_'+'sum']+0.00001)
        

In [17]:
# # 테스트중 

# for g_fun in ['median']:
    
#     for fun_name in ['_minus_','_divide_']:

#         a1 = [
#              'ZN_divide_ZN_YEAR_COMPONENT_median',
#              'FE_divide_FE_YEAR_COMPONENT_median',
#              'V40_divide_V40_YEAR_COMPONENT_median',
#              'CU_divide_CU_YEAR_COMPONENT_median',
#              'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_median',
#              'ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_COMPONENT_median'
#         ]

#         train[g_fun+fun_name+'diff_g_std'] = train[a1].apply(lambda x:x.std(),axis=1)
#         test[g_fun+fun_name+'diff_g_std'] = test[a1].apply(lambda x:x.std(),axis=1)

In [18]:
"""
count encoding
"""

opt_numeric_var = ['ANONYMOUS_1_V40', 'ANONYMOUS_1_CR', 'ANONYMOUS_1_ZN','ANONYMOUS_1_FE']

for group in ['COMPONENT_ARBITRARY','YEAR_COMPONENT','YEAR']+['FE_woe_bin']:    
    
    for numeric_var in ['ZN','FE','V40','PQINDEX','CU','ANONYMOUS_1','ANONYMOUS_2']+opt_numeric_var:
        
        train = train.copy()
        test = test.copy()

        a1 = train.groupby([group])[numeric_var].value_counts().to_dict()    
        train[var+'_cnt'] = train[var].map(a1)
        test[var+'_cnt'] = test[var].map(a1)
        
        a1 = train.groupby([group])[numeric_var].nunique().to_dict()
        train[var+'_nunique'] = train[var].map(a1)
        test[var+'_nunique'] = test[var].map(a1)

In [19]:
"""
clustetring features
"""

np.random.seed(1)

features = ['ZN','FE','V40','CU','MO']

# sc = MinMaxScaler((0, 1))
# sc_fit = sc.fit(train[features])
# train_scaled = sc_fit.transform(train[features])
# kmeans = KMeans(n_clusters=15,n_init=25, max_iter = 600, random_state=1).fit(train_scaled)
# test_scaled = sc_fit.transform(test[features])
# train["cluster_no"] = kmeans.predict(train_scaled)
# test["cluster_no"] = kmeans.predict(test_scaled)

# 앞에서 min-max 이미 적용 함 
kmeans = KMeans(n_clusters=15,n_init=25, max_iter = 600, random_state=1).fit(train[features])
train["cluster_no"] = kmeans.predict(train[features])
test["cluster_no"] = kmeans.predict(test[features])

# check
# print(train['cluster_no'].value_counts())

for var in ['ANONYMOUS_1','ANONYMOUS_2']+['ANONYMOUS_1_V40', 'ANONYMOUS_1_CR', 'ANONYMOUS_1_ZN','ANONYMOUS_1_FE']:
    
    a1 = train.groupby(['cluster_no'])[var].mean().to_dict()
    train['cluster_no_by'+'_'+var+'_'+'mean'] = train['cluster_no'].map(a1)
    test['cluster_no_by'+'_'+var+'_'+'mean'] = test['cluster_no'].map(a1)
    
    a1 = train.groupby(['cluster_no'])[var].median().to_dict()
    train['cluster_no_by'+'_'+var+'_'+'median'] = train['cluster_no'].map(a1)
    test['cluster_no_by'+'_'+var+'_'+'median'] = test['cluster_no'].map(a1)

    a1 = train.groupby(['cluster_no'])[var].max().to_dict()
    train['cluster_no_by'+'_'+var+'_'+'max'] = train['cluster_no'].map(a1)
    test['cluster_no_by'+'_'+var+'_'+'max'] = test['cluster_no'].map(a1)

    a1 = train.groupby(['cluster_no'])[var].min().to_dict()
    train['cluster_no_by'+'_'+var+'_'+'min'] = train['cluster_no'].map(a1)
    test['cluster_no_by'+'_'+var+'_'+'min'] = test['cluster_no'].map(a1)

In [20]:
# check inf values 
a1 = train.select_dtypes(include=['int','float']).apply(lambda x: x.max(),axis=0).reset_index(name='val')
a1.loc[a1['val']==np.Inf,'index'].tolist()

[]

In [21]:
"""
addtional data 

train_vote = pd.read_csv('/kaggle/input/traintestvote/train_vote.csv')
test_vote = pd.read_csv('/kaggle/input/traintestvote/test_vote.csv')

a1 = test_vote.loc[test_vote['vote']>30,:]
additional_train1 = test.loc[test['ID'].isin(a1['ID'].tolist()),:].reset_index(drop=True)
additional_train1['Y_LABEL'] = 1
print(len(additional_train1))

# a1 = test_vote.loc[test_vote['vote']==0,:]
# additional_train0 = test.loc[test['ID'].isin(a1['ID'].tolist()),:].reset_index(drop=True)
# additional_train0['Y_LABEL'] = 0
# print(len(additional_train0))

train = pd.concat([train,additional_train1],axis=0).reset_index(drop=True)
"""

"\naddtional data \n\ntrain_vote = pd.read_csv('/kaggle/input/traintestvote/train_vote.csv')\ntest_vote = pd.read_csv('/kaggle/input/traintestvote/test_vote.csv')\n\na1 = test_vote.loc[test_vote['vote']>30,:]\nadditional_train1 = test.loc[test['ID'].isin(a1['ID'].tolist()),:].reset_index(drop=True)\nadditional_train1['Y_LABEL'] = 1\nprint(len(additional_train1))\n\n# a1 = test_vote.loc[test_vote['vote']==0,:]\n# additional_train0 = test.loc[test['ID'].isin(a1['ID'].tolist()),:].reset_index(drop=True)\n# additional_train0['Y_LABEL'] = 0\n# print(len(additional_train0))\n\ntrain = pd.concat([train,additional_train1],axis=0).reset_index(drop=True)\n"

> 모델학습

In [22]:
def train_model(train,test,params,stratified,num_folds,drop_features,seed_num):
    
    # start log 
    print('-'*50)
    print('>> seed_num:',seed_num)   
    print('>> drop_features:',len(drop_features))
    
    seed_everything(1)
    
    # Divide in training/validation and test data
    train_df = train.copy()
    test_df = test.copy()

    # label encoding 
    encoder = LabelEncoder()
    categorical_features = [i for i in train_df.select_dtypes(include=['object','category']).columns.tolist() if i not in ['ID']]
    categorical_features = [i for i in categorical_features if i in train_df.columns.tolist()]
    for each in categorical_features:
        train_df[each] = encoder.fit_transform(train_df[each])
        test_df[each] = encoder.fit_transform(test_df[each])

    # set training options
    stratified = stratified
    num_folds = num_folds

    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=1)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=1)

    # Create arrays and dataframes to store results
    oof_preds_lgb = np.zeros(train_df.shape[0])
    sub_preds_lgb = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['Y_LABEL','ID','SAMPLE_TRANSFER_DAY']+drop_features]

    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['Y_LABEL'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['Y_LABEL'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['Y_LABEL'].iloc[valid_idx]

        """        
        learning_rate=0.02,
        num_leaves=34,
        colsample_bytree=0.2,
        subsample=0.8715623,
        max_depth=4,
        reg_alpha=0.041545473,
        reg_lambda=0.0735294,
        min_split_gain=0.0222415,
        min_child_weight=39.3259775,
        min_child_samples = 10,
        """

        # LightGBM parameters found by Bayesian optimization
        clf = LGBMClassifier(
            
            learning_rate = params['learning_rate'],
            num_leaves = int(round(params['num_leaves'])),
            colsample_bytree = params['colsample_bytree'],
            subsample = params['subsample'],
            max_depth = int(round(params['max_depth'])),
            reg_alpha = params['reg_alpha'],
            reg_lambda = params['reg_lambda'],
            min_split_gain = params['min_split_gain'],
            min_child_weight = params['min_child_weight'],
            min_child_samples = int(round(params['min_child_samples'])),    
            
            n_jobs = -1,
            n_estimators = 10000,            
            random_state = seed_num,
            silent=-1,
            deterministic=True,
            verbose=-1
        )
        
        with warnings.catch_warnings():
            
            warnings.filterwarnings('ignore')

            clf.fit(
                  train_x
                , train_y
                , eval_set=[(train_x, train_y), (valid_x, valid_y)]
                , eval_metric= 'auc'
                , verbose= -1
                , early_stopping_rounds= 500
            )

        oof_preds_lgb[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
        sub_preds_lgb += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds_lgb[valid_idx])))

    print('Full AUC score %.6f' % roc_auc_score(train_df['Y_LABEL'], oof_preds_lgb))

    # Write submission file and plot feature importance
    test_df['Y_LABEL_lgb'] = sub_preds_lgb

    # vi
    # print('-'*50)
    display(feature_importance_df.groupby(['feature'])['importance'].sum().sort_values(ascending=False).head(30))
    # print('-'*50)
    # display_importances(feature_importance_df)
    
    # train auc
    oof_auc = roc_auc_score(train_df['Y_LABEL'], oof_preds_lgb)

    
    if oof_auc>=0.701:

        # find the best thred for f1-score
        f1_score_df = pd.DataFrame()
        for thred in [i/10000 for i in range(0,10000,1) if (i/10000>0.1) & (i/10000<0.35)]:

            a1 = pd.DataFrame()
            f1 = f1_score(train_df['Y_LABEL'], np.where(oof_preds_lgb>thred,1,0), average='macro')
            a1['f1'] = [f1]
            a1['thred'] = [thred]
            f1_score_df = pd.concat([f1_score_df, a1], axis=0)

        thred = f1_score_df.loc[f1_score_df['f1']==f1_score_df['f1'].max(),'thred'].tolist()[0]
        print('thred:',thred)
        print('ncol',len(feats))

        # train f1
        print('auc:',oof_auc)
        oof_f1 = f1_score(train_df['Y_LABEL'], np.where(oof_preds_lgb>thred,1,0), average='macro')
        print('f1:',oof_f1)
        a1 = train_df['Y_LABEL'].value_counts()/len(train_df)
        print('Target ratio(real):',(a1[1]))

        # test err
        test_df['TARGET'] = np.where(test_df['Y_LABEL_lgb']>thred,1,0)
        a1 = test_df['TARGET'].value_counts()/len(test_df)
        print('Target ratio(pred):',(a1[1]))
        target_sum = test_df['TARGET'].sum()
        print('Target sum:',target_sum)        
        
        # save 
        train_df['Y_LABEL_lgb'] = oof_preds_lgb
        a1 = train_df[['ID','YEAR','COMPONENT_ARBITRARY','Y_LABEL_lgb','Y_LABEL']].copy()
        a1.to_csv('train_pred_'+str(seed_num)+'_'+str(np.round(oof_f1,10))+'.csv', index= False)    
        a1 = test_df[['ID','YEAR','COMPONENT_ARBITRARY','Y_LABEL_lgb']].copy()
        a1.to_csv('test_pred_'+str(seed_num)+'_'+str(np.round(oof_f1,10))+'.csv', index= False)

        # submit
        a1 = test_df[['ID', 'TARGET']].copy()
        a1 = a1.rename(columns={'TARGET':'Y_LABEL'})
        submission_file_name = 'sample_submission_lgb_'+str(np.round(oof_f1,4))+'.csv'
        a1.to_csv(submission_file_name, index= False)

In [23]:
"""
baseline model 
"""

drop_features_vc1 = ['ZN_minus_ZN_YEAR_COMPONENT_median', 'ANONYMOUS_1_CU', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_COMPONENT_min', 'PQINDEX_divide_PQINDEX_YEAR_min', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_FE_woe_bin_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_min', 'V40_minus_V40_FE_woe_bin_sum', 'PQINDEX_minus_PQINDEX_FE_woe_bin_sum', 'V40_minus_V40_YEAR_COMPONENT_min', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_median', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_sum', 'FE_divide_FE_YEAR_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_COMPONENT_ARBITRARY_median', 'PQINDEX_minus_PQINDEX_YEAR_COMPONENT_max', 'PQINDEX_minus_PQINDEX_YEAR_COMPONENT_median', 'FE_minus_FE_YEAR_COMPONENT_median', 'ZN_divide_ZN_YEAR_min', 'PQINDEX_divide_PQINDEX_YEAR_median', 'FE_minus_FE_YEAR_COMPONENT_max', 'ZN_divide_ZN_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_FE_woe_bin_median', 'ANONYMOUS_1_ZN_YEAR_COMPONENT_max', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_COMPONENT_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_COMPONENT_sum', 'ANONYMOUS_1_FE_YEAR_median', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_FE_woe_bin_sum', 'CU_divide_CU_COMPONENT_ARBITRARY_sum', 'FE_divide_FE_YEAR_median', 'ANONYMOUS_2_YEAR_max', 'FE_divide_FE_FE_woe_bin_min', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_FE_woe_bin_sum', 'V40_minus_V40_YEAR_COMPONENT_max', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_sum', 'V40_divide_V40_FE_woe_bin_min', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_COMPONENT_median', 'ZN_minus_ZN_YEAR_COMPONENT_min', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_COMPONENT_max', 'V40_YEAR_median', 'V40_minus_V40_YEAR_sum', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_max', 'NI_V40', 'V40_minus_V40_COMPONENT_ARBITRARY_min', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_max', 'ZN_minus_ZN_YEAR_min', 'FE_divide_FE_YEAR_COMPONENT_max', 'ZN_divide_ZN_FE_woe_bin_min', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_FE_woe_bin_median', 'CU_minus_CU_YEAR_COMPONENT_median', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_max', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_FE_woe_bin_sum', 'ZN_divide_ZN_YEAR_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_FE_woe_bin_max', 'ANONYMOUS_2_divide_ANONYMOUS_2_FE_woe_bin_max', 'ZN_minus_ZN_FE_woe_bin_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_COMPONENT_ARBITRARY_max', 'FE_minus_FE_YEAR_max', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_COMPONENT_ARBITRARY_min', 'CU_YEAR_COMPONENT_median', 'cluster_no_by_ANONYMOUS_1_FE_median', 'ZN_YEAR_COMPONENT_sum', 'CU_minus_CU_FE_woe_bin_max', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_COMPONENT_median', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_median', 'CU_divide_CU_YEAR_COMPONENT_max', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_FE_woe_bin_max', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_YEAR_COMPONENT_min', 'CU_divide_CU_YEAR_max', 'ZN_divide_ZN_COMPONENT_ARBITRARY_max', 'FE_divide_FE_FE_woe_bin_max', 'cluster_no_by_ANONYMOUS_1_median', 'V40_divide_V40_FE_woe_bin_sum', 'V40_divide_V40_COMPONENT_ARBITRARY_max', 'ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_COMPONENT_sum', 'FE_YEAR_COMPONENT_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_COMPONENT_ARBITRARY_median', 'FE_divide_FE_COMPONENT_ARBITRARY_median', 'V40_divide_V40_YEAR_COMPONENT_max', 'V40_minus_V40_YEAR_min', 'ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_median', 'FE_minus_FE_FE_woe_bin_median', 'FE_minus_FE_YEAR_median', 'ANONYMOUS_1_FE_YEAR_COMPONENT_sum', 'ZN_minus_ZN_FE_woe_bin_sum', 'CU_minus_CU_FE_woe_bin_sum', 'V40', 'ANONYMOUS_1_ZN_woe_bin', 'FE_divide_FE_COMPONENT_ARBITRARY_min', 'CU_divide_CU_FE_woe_bin_sum', 'cluster_no_by_ANONYMOUS_2_mean', 'CU', 'V40_minus_V40_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_minus_ANONYMOUS_1_FE_woe_bin_median', 'V40_divide_V40_YEAR_min', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_COMPONENT_ARBITRARY_sum', 'ZN_minus_ZN_YEAR_COMPONENT_max', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_median', 'cluster_no_by_ANONYMOUS_1_V40_mean', 'FE_minus_FE_FE_woe_bin_min', 'PQINDEX_divide_PQINDEX_YEAR_max', 'ANONYMOUS_2_minus_ANONYMOUS_2_FE_woe_bin_sum', 'FE_divide_FE_YEAR_sum', 'V40_minus_V40_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_FE_woe_bin_max', 'PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_sum', 'ANONYMOUS_1_V40', 'ZN_divide_ZN_YEAR_sum', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_COMPONENT_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_FE_woe_bin_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_COMPONENT_ARBITRARY_sum', 'PQINDEX_minus_PQINDEX_FE_woe_bin_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_max', 'PQINDEX_divide_PQINDEX_FE_woe_bin_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_sum', 'ZN_divide_ZN_YEAR_median', 'ZN_minus_ZN_YEAR_COMPONENT_sum', 'ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_COMPONENT_median', 'V40_minus_V40_YEAR_max', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_FE_woe_bin_min', 'CR_FE', 'FE_V40', 'FE_divide_FE_YEAR_COMPONENT_min', 'V40_divide_V40_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE', 'FE_divide_FE_COMPONENT_ARBITRARY_max', 'PQINDEX_minus_PQINDEX_YEAR_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_median', 'ZN_divide_ZN_YEAR_COMPONENT_median', 'ANONYMOUS_1_minus_ANONYMOUS_1_COMPONENT_ARBITRARY_max', 'CU_divide_CU_YEAR_median', 'ZN_divide_ZN_YEAR_COMPONENT_min', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_FE_woe_bin_max', 'PQINDEX_divide_PQINDEX_FE_woe_bin_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_COMPONENT_max', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_median', 'ANONYMOUS_1_MO', 'CU_FE', 'ZN_minus_ZN_YEAR_sum', 'ZN_minus_ZN_COMPONENT_ARBITRARY_max', 'V40_divide_V40_FE_woe_bin_max', 'PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_FE_woe_bin_sum', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_median', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_YEAR_min', 'PQINDEX_divide_PQINDEX_FE_woe_bin_min', 'PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_min', 'ANONYMOUS_2_divide_ANONYMOUS_2_COMPONENT_ARBITRARY_median', 'PQINDEX_YEAR_COMPONENT_sum', 'PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_median', 'PQINDEX_minus_PQINDEX_YEAR_COMPONENT_sum', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_max', 'CU_ZN', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_COMPONENT_max', 'ZN_divide_ZN_YEAR_COMPONENT_max', 'V40_minus_V40_FE_woe_bin_min', 'NI_ZN', 'V40_divide_V40_FE_woe_bin_median', 'ZN_minus_ZN_YEAR_max', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_median', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_max', 'ANONYMOUS_1_ZN', 'PQINDEX_YEAR_median', 'NI', 'FE_ZN', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_max', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_FE_woe_bin_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_median', 'CR_V40', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_COMPONENT_median', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_min', 'ANONYMOUS_2_divide_ANONYMOUS_2_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_COMPONENT_ARBITRARY_min', 'MO_V40', 'PQINDEX_minus_PQINDEX_FE_woe_bin_max', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_min', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_median', 'FE_minus_FE_YEAR_COMPONENT_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_max', 'cluster_no_by_ANONYMOUS_1_V40_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_max', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_COMPONENT_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_min', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_FE_woe_bin_median', 'ANONYMOUS_2_MO', 'PQINDEX_log1', 'CU_minus_CU_YEAR_COMPONENT_max', 'FE_NI', 'ANONYMOUS_1_divide_ANONYMOUS_1_FE_woe_bin_max', 'CU_divide_CU_FE_woe_bin_median']
drop_features_vc2 = ['PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_median', 'ZN_minus_ZN_YEAR_COMPONENT_min', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_median', 'PQINDEX_divide_PQINDEX_YEAR_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_max', 'FE_divide_FE_FE_woe_bin_min', 'V40_divide_V40_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_FE_woe_bin_sum', 'FE_divide_FE_YEAR_COMPONENT_sum', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_median', 'FE_divide_FE_COMPONENT_ARBITRARY_max', 'CU_divide_CU_YEAR_max', 'FE_divide_FE_YEAR_sum', 'V40_divide_V40_YEAR_min', 'FE_divide_FE_FE_woe_bin_max', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_COMPONENT_sum', 'ANONYMOUS_2_divide_ANONYMOUS_2_FE_woe_bin_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_COMPONENT_sum', 'V40_divide_V40_FE_woe_bin_sum', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE_YEAR_median', 'ZN_divide_ZN_YEAR_min', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_min', 'ZN_minus_ZN_FE_woe_bin_sum', 'PQINDEX_divide_PQINDEX_YEAR_min', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_FE_woe_bin_min', 'CU_divide_CU_YEAR_COMPONENT_min', 'ANONYMOUS_2_YEAR_max', 'CU_minus_CU_FE_woe_bin_sum', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_max', 'FE_minus_FE_FE_woe_bin_sum', 'FE_divide_FE_YEAR_max', 'PQINDEX_minus_PQINDEX_FE_woe_bin_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_sum', 'FE_minus_FE_YEAR_max', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_COMPONENT_min', 'FE_minus_FE_FE_woe_bin_min', 'cluster_no_by_ANONYMOUS_1_FE_median', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_COMPONENT_median', 'ZN_minus_ZN_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_FE_woe_bin_sum', 'ZN_minus_ZN_YEAR_sum', 'NI_V40', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_max', 'CU_minus_CU_YEAR_sum', 'V40_minus_V40_FE_woe_bin_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_median', 'PQINDEX_divide_PQINDEX_FE_woe_bin_max', 'ZN_minus_ZN_FE_woe_bin_max', 'V40_minus_V40_COMPONENT_ARBITRARY_min', 'FE_minus_FE_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE', 'PQINDEX_minus_PQINDEX_FE_woe_bin_median', 'PQINDEX_minus_PQINDEX_YEAR_max', 'FE_YEAR_COMPONENT_median', 'V40_divide_V40_COMPONENT_ARBITRARY_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_sum', 'ZN_minus_ZN_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_CR', 'CU_minus_CU_YEAR_COMPONENT_max', 'CU_minus_CU_FE_woe_bin_max', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_COMPONENT_max', 'FE', 'V40_minus_V40_YEAR_min', 'ZN_divide_ZN_YEAR_COMPONENT_max', 'V40_divide_V40_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_FE_woe_bin_min', 'V40_divide_V40_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_max', 'CU_divide_CU_YEAR_COMPONENT_max', 'NI_ZN', 'ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_COMPONENT_sum', 'FE_YEAR_COMPONENT_max', 'FE_divide_FE_YEAR_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_FE_woe_bin_median', 'ZN_divide_ZN_YEAR_COMPONENT_min', 'CU_divide_CU_YEAR_COMPONENT_sum', 'FE_minus_FE_FE_woe_bin_median', 'ANONYMOUS_2_divide_ANONYMOUS_2_FE_woe_bin_sum', 'CU_divide_CU_FE_woe_bin_sum', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_COMPONENT_ARBITRARY_median', 'FE_divide_FE_YEAR_COMPONENT_min', 'V40_minus_V40_FE_woe_bin_sum', 'ANONYMOUS_2_divide_ANONYMOUS_2_COMPONENT_ARBITRARY_max', 'V40_divide_V40_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_median', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_FE_woe_bin_sum', 'PQINDEX_minus_PQINDEX_FE_woe_bin_sum', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_COMPONENT_max', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_sum', 'CR_ZN', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_max', 'CR_V40', 'V40_minus_V40_FE_woe_bin_max', 'V40_minus_V40_YEAR_COMPONENT_sum', 'ZN_divide_ZN_COMPONENT_ARBITRARY_max', 'CU_divide_CU_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_FE_woe_bin_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_min', 'ZN_minus_ZN_YEAR_COMPONENT_max', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_median', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_COMPONENT_sum', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_min', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_median', 'FE_minus_FE_YEAR_median', 'ANONYMOUS_1_minus_ANONYMOUS_1_COMPONENT_ARBITRARY_min', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_max', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_COMPONENT_ARBITRARY_max', 'V40_divide_V40_YEAR_max', 'ANONYMOUS_2_MO', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_YEAR_COMPONENT_sum', 'V40_divide_V40_YEAR_COMPONENT_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_COMPONENT_ARBITRARY_min', 'PQINDEX_divide_PQINDEX_FE_woe_bin_sum', 'ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_COMPONENT_median', 'MO_NI', 'MO_ZN', 'ANONYMOUS_1_minus_ANONYMOUS_1_FE_woe_bin_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_median', 'ZN_minus_ZN_YEAR_COMPONENT_median', 'CR_MO', 'CR_FE', 'ZN_divide_ZN_FE_woe_bin_max', 'V40_minus_V40_YEAR_COMPONENT_max', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_median', 'FE_divide_FE_YEAR_min', 'PQINDEX_minus_PQINDEX_YEAR_COMPONENT_median', 'V40_woe_bin', 'PQINDEX_divide_PQINDEX_FE_woe_bin_min', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_median', 'CU_YEAR_COMPONENT_max', 'ANONYMOUS_2_YEAR_COMPONENT_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_FE_woe_bin_min', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_FE_woe_bin_max', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_FE_woe_bin_median', 'ZN_minus_ZN_FE_woe_bin_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_sum', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_min', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_sum', 'FE_NI', 'ANONYMOUS_1_divide_ANONYMOUS_1_FE_woe_bin_sum', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_YEAR_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_COMPONENT_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_sum', 'NI', 'ZN_minus_ZN_YEAR_COMPONENT_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_FE_woe_bin_max', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_max', 'ZN_divide_ZN_YEAR_COMPONENT_median', 'V40_YEAR_median', 'PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_min', 'V40_divide_V40_FE_woe_bin_median', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_COMPONENT_max', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_FE_woe_bin_min', 'ANONYMOUS_1_CU', 'CU_divide_CU_FE_woe_bin_max', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_FE_woe_bin_max', 'V40', 'ANONYMOUS_1_divide_ANONYMOUS_1_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_log1', 'ANONYMOUS_2_minus_ANONYMOUS_2_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_max', 'V40_divide_V40_FE_woe_bin_min', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_FE_woe_bin_sum']
drop_features_vc3 = ['PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_median', 'ZN_minus_ZN_YEAR_COMPONENT_min', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_median', 'PQINDEX_divide_PQINDEX_YEAR_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_max', 'FE_divide_FE_FE_woe_bin_min', 'V40_divide_V40_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_FE_woe_bin_sum', 'FE_divide_FE_YEAR_COMPONENT_sum', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_median', 'FE_divide_FE_COMPONENT_ARBITRARY_max', 'CU_divide_CU_YEAR_max', 'FE_divide_FE_YEAR_sum', 'V40_divide_V40_YEAR_min', 'FE_divide_FE_FE_woe_bin_max', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_COMPONENT_sum', 'ANONYMOUS_2_divide_ANONYMOUS_2_FE_woe_bin_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_COMPONENT_sum', 'V40_divide_V40_FE_woe_bin_sum', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE_YEAR_median', 'ZN_divide_ZN_YEAR_min', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_min', 'ZN_minus_ZN_FE_woe_bin_sum', 'PQINDEX_divide_PQINDEX_YEAR_min', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_FE_woe_bin_min', 'CU_divide_CU_YEAR_COMPONENT_min', 'ANONYMOUS_2_YEAR_max', 'CU_minus_CU_FE_woe_bin_sum', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_max', 'FE_minus_FE_FE_woe_bin_sum', 'FE_divide_FE_YEAR_max', 'PQINDEX_minus_PQINDEX_FE_woe_bin_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_sum', 'FE_minus_FE_YEAR_max', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_COMPONENT_min', 'FE_minus_FE_FE_woe_bin_min', 'cluster_no_by_ANONYMOUS_1_FE_median', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_COMPONENT_median', 'ZN_minus_ZN_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_FE_woe_bin_sum', 'ZN_minus_ZN_YEAR_sum', 'NI_V40', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_max', 'CU_minus_CU_YEAR_sum', 'V40_minus_V40_FE_woe_bin_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_median', 'PQINDEX_divide_PQINDEX_FE_woe_bin_max', 'ZN_minus_ZN_FE_woe_bin_max', 'V40_minus_V40_COMPONENT_ARBITRARY_min', 'FE_minus_FE_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE', 'PQINDEX_minus_PQINDEX_FE_woe_bin_median', 'PQINDEX_minus_PQINDEX_YEAR_max', 'FE_YEAR_COMPONENT_median', 'V40_divide_V40_COMPONENT_ARBITRARY_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_sum', 'ZN_minus_ZN_COMPONENT_ARBITRARY_max']
drop_features_vc4 = ['PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_median', 'ZN_minus_ZN_YEAR_COMPONENT_min', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_median', 'PQINDEX_divide_PQINDEX_YEAR_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_max', 'FE_divide_FE_FE_woe_bin_min', 'V40_divide_V40_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_max']

params =  {
    'learning_rate': 0.07398,
    'max_depth': 4.309,
    'colsample_bytree': 0.4028,
    'subsample': 0.4278,
    'min_child_samples': 25.65,
    'min_child_weight': 0.6138,
    'min_split_gain': 0.7354,
    'num_leaves': 62.68,
    'reg_alpha': 0.2889,
    'reg_lambda': 7.875
}

# 176
# for i in [18,22,82,113,119,155,176,195,196]:
train_model(train,test,params,True,5,drop_features_vc1,seed_num=176)

--------------------------------------------------
>> seed_num: 176
>> drop_features: 195
Fold  1 AUC : 0.699257
Fold  2 AUC : 0.718767
Fold  3 AUC : 0.699679
Fold  4 AUC : 0.716366
Fold  5 AUC : 0.691165
Full AUC score 0.704596


feature
ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_YEAR_COMPONENT_median    92
FE_minus_FE_YEAR_sum                                            82
ZN_minus_ZN_YEAR_median                                         82
CU_divide_CU_YEAR_sum                                           72
FE_minus_FE_FE_woe_bin_max                                      71
ZN_minus_ZN_COMPONENT_ARBITRARY_median                          69
PQINDEX_minus_PQINDEX_YEAR_sum                                  67
ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_sum               66
ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_median            60
ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_max                   59
FE_divide_FE_FE_woe_bin_median                                  55
ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_COMPONENT_max               55
YEAR_COMPONENT                                                  53
ZN_divide_ZN_FE_woe_bin_median                                  53
CU_divide_CU_YEAR_COMPONENT_median                    

thred: 0.1636
ncol 430
auc: 0.7045962957432151
f1: 0.5922652586883465
Target ratio(real): 0.08534941468605889
Target ratio(pred): 0.08872703194835292
Target sum: 536


In [24]:
"""
--------------------------------------------------
>> seed_num: 176
>> drop_features: 195
Fold  1 AUC : 0.699257
Fold  2 AUC : 0.718767
Fold  3 AUC : 0.699679
Fold  4 AUC : 0.716366
Fold  5 AUC : 0.691165
Full AUC score 0.704596
feature
ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_YEAR_COMPONENT_median    92
FE_minus_FE_YEAR_sum                                            82
ZN_minus_ZN_YEAR_median                                         82
CU_divide_CU_YEAR_sum                                           72
FE_minus_FE_FE_woe_bin_max                                      71
ZN_minus_ZN_COMPONENT_ARBITRARY_median                          69
PQINDEX_minus_PQINDEX_YEAR_sum                                  67
ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_sum               66
ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_median            60
ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_max                   59
FE_divide_FE_FE_woe_bin_median                                  55
ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_COMPONENT_max               55
YEAR_COMPONENT                                                  53
ZN_divide_ZN_FE_woe_bin_median                                  53
CU_divide_CU_YEAR_COMPONENT_median                              53
CR_MO                                                           53
V40_divide_V40_YEAR_COMPONENT_min                               51
CU_minus_CU_YEAR_sum                                            50
ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_COMPONENT_max         50
FE_divide_FE_YEAR_COMPONENT_sum                                 50
ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_median                       48
ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_COMPONENT_sum         47
ANONYMOUS_1_divide_ANONYMOUS_1_COMPONENT_ARBITRARY_median       45
ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_YEAR_median              45
ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_median                      44
ANONYMOUS_1_minus_ANONYMOUS_1_COMPONENT_ARBITRARY_median        43
FE_divide_FE_YEAR_COMPONENT_median                              42
FE_minus_FE_FE_woe_bin_sum                                      41
ZN_divide_ZN_FE_woe_bin_sum                                     40
PQINDEX_minus_PQINDEX_YEAR_max                                  40
Name: importance, dtype: int32
thred: 0.1636
ncol 430
auc: 0.7045962957432151
f1: 0.5922652586883465
Target ratio(real): 0.08534941468605889
Target ratio(pred): 0.08872703194835292
Target sum: 536
"""

'\n--------------------------------------------------\n>> seed_num: 176\n>> drop_features: 195\nFold  1 AUC : 0.699257\nFold  2 AUC : 0.718767\nFold  3 AUC : 0.699679\nFold  4 AUC : 0.716366\nFold  5 AUC : 0.691165\nFull AUC score 0.704596\nfeature\nANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_YEAR_COMPONENT_median    92\nFE_minus_FE_YEAR_sum                                            82\nZN_minus_ZN_YEAR_median                                         82\nCU_divide_CU_YEAR_sum                                           72\nFE_minus_FE_FE_woe_bin_max                                      71\nZN_minus_ZN_COMPONENT_ARBITRARY_median                          69\nPQINDEX_minus_PQINDEX_YEAR_sum                                  67\nANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_sum               66\nANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_median            60\nANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_max                   59\nFE_divide_FE_FE_woe_bin_median                                  55\nAN

In [25]:
def bayes_parameter_opt_lgb(
    train, 
    opt_params, 
    drop_features,
    init_round=15, 
    opt_round=25, 
    n_folds=3, 
    random_seed=1, 
    n_estimators=10000, 
    output_process=False
    ):   
    
    seed_everything(1)
    
    train_df = train.copy()

    # label encoding 
    encoder = LabelEncoder()
    categorical_features = [i for i in train_df.select_dtypes(include=['object','category']).columns.tolist() if i not in ['ID']]
    categorical_features = [i for i in categorical_features if i in train_df.columns.tolist()]
    for each in categorical_features:
        train_df[each] = encoder.fit_transform(train_df[each])
        
    # feats = [f for f in train_df.columns if f not in ['Y_LABEL','ID','SAMPLE_TRANSFER_DAY']]    
    # X = train_df[feats].copy()
    # y = train_df['Y_LABEL'].copy()
    
    # prepare data
    # train_data = lgb.Dataset(data=X, label=y, free_raw_data=False)
    
    # parameters
    def lgb_eval(
          learning_rate
        , num_leaves
        , colsample_bytree
        , subsample
        , max_depth
        , reg_alpha
        , reg_lambda
        , min_split_gain
        , min_child_weight
        , min_child_samples
        , drop_features=drop_features

    ):
        
        params = {'application':'binary', 'metric':'auc'}
        params['learning_rate'] = max(min(learning_rate, 1), 0)
        params["num_leaves"] = int(round(num_leaves))
        params['colsample_bytree'] = max(min(colsample_bytree, 1), 0)
        params['subsample'] = max(min(subsample, 1), 0)
        params['max_depth'] = int(round(max_depth))        
        params['reg_alpha'] = reg_alpha
        params['reg_lambda'] = reg_lambda        
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        params['min_child_samples'] = int(round(min_child_samples))
               
        # -----        
        
        stratified = True
        
        # Cross validation model
        if stratified:
            folds = StratifiedKFold(n_splits= 5, shuffle=True, random_state=1)
        else:
            folds = KFold(n_splits= 5, shuffle=True, random_state=1)

        # Create arrays and dataframes to store results
        oof_preds_lgb = np.zeros(train_df.shape[0])

        feats = [f for f in train_df.columns if f not in ['Y_LABEL','ID','SAMPLE_TRANSFER_DAY']+drop_features]
        # print('drop_features',len(drop_features))
        # print('feats',len(feats))

        for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['Y_LABEL'])):
            train_x, train_y = train_df[feats].iloc[train_idx], train_df['Y_LABEL'].iloc[train_idx]
            valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['Y_LABEL'].iloc[valid_idx]

            # LightGBM parameters found by Bayesian optimization
            clf = LGBMClassifier(

                **params,

                n_jobs = -1,
                n_estimators = 10000,            
                random_state = random_seed,
                silent=True,
                deterministic=True,
                verbose=-100
            )

            with warnings.catch_warnings():

                warnings.filterwarnings('ignore')

                clf.fit(
                      train_x
                    , train_y
                    , eval_set=[(train_x, train_y), (valid_x, valid_y)]
                    , eval_metric= 'auc'
                    , verbose= False
                    , early_stopping_rounds= 500
                )

            oof_preds_lgb[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]

        cv_result = roc_auc_score(train_df['Y_LABEL'], oof_preds_lgb)        
       
        # ----

        return cv_result

    lgbBO = BayesianOptimization(lgb_eval, opt_params, random_state=random_seed)
    
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore')
        lgbBO.maximize(init_points=init_round, n_iter=opt_round, acq='ucb')
    
    model_auc=[]
    for model in range(len( lgbBO.res)):
        model_auc.append(lgbBO.res[model]['target'])
    
    a1 = lgbBO.res[pd.Series(model_auc).idxmax()]['target'],lgbBO.res[pd.Series(model_auc).idxmax()]['params']
    file_name = 'res_tune_'+str(len(drop_features))+'_'+str(a1[0])+'.joblib'    
    joblib.dump(a1[1],file_name)
    
    return a1

In [26]:
def num_feature(df,num_features):
    if num_features[0][:5] == 'rank_':
        num_agg_df = df.groupby("YEAR",sort=False)[num_features].agg(['last'])
    else:
        num_agg_df = df.groupby("YEAR",sort=False)[num_features].agg(['mean', 'std', 'min', 'max', 'sum'])
    num_agg_df.columns = ['_'.join(x) for x in num_agg_df.columns]
    if num_features[0][:5] != 'rank_':
        for col in num_agg_df.columns:
            num_agg_df[col] = num_agg_df[col] // 0.01
    df = num_agg_df.reset_index()
    print('num feature shape after engineering', df.shape )

    return df


num_features13 = ['AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI',  'TI', 'V', 'V40', 'ZN']
num_features3 = ['ANONYMOUS_1', 'ANONYMOUS_2','PQINDEX'] 
num_features = num_features13+num_features3

a1 = num_feature(train,num_features)
train = train.merge(a1,on=['YEAR'])
a1 = num_feature(test,num_features)
test = test.merge(a1,on=['YEAR'])

num feature shape after engineering (16, 81)
num feature shape after engineering (16, 81)


In [27]:
def diff_feature(df):
    diff_num_features = [f'diff_{col}' for col in num_features]
    cids = df['YEAR'].values
    df = df.groupby('YEAR')[num_features].diff().add_prefix('diff_')
    df.insert(0,'YEAR',cids)
    num_agg_df = df.groupby("YEAR",sort=False)[diff_num_features].agg(['mean', 'std', 'min', 'max', 'sum'])
    num_agg_df.columns = ['_'.join(x) for x in num_agg_df.columns]
    for col in num_agg_df.columns:
        num_agg_df[col] = num_agg_df[col] // 0.01

    df = num_agg_df.reset_index()
    print('diff feature shape after engineering', df.shape )

    return df

num_features13 = ['AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI',  'TI', 'V', 'V40', 'ZN']
num_features3 = ['ANONYMOUS_1', 'ANONYMOUS_2','PQINDEX'] 
num_features = num_features13+num_features3

a1 = diff_feature(train)
train = train.merge(a1,on=['YEAR'])
a1 = diff_feature(test)
test = test.merge(a1,on=['YEAR'])

diff feature shape after engineering (16, 81)
diff feature shape after engineering (16, 81)


In [28]:
all_num_features = [i for i in train.select_dtypes(include=[int,float]).columns if i not in ['Y_LABEL']]
cor_matrix = train[all_num_features].corr().abs()
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
print('to_drop:',len(to_drop))
train = train.drop(columns=to_drop, axis=1)
test = test.drop(columns=to_drop, axis=1)

to_drop: 383


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


In [29]:
print(len(train.columns))

405


In [30]:
%%time 

"""
모델 튜닝 

// reference params
'learning_rate': 0.07398,
'max_depth': 4.309,
'colsample_bytree': 0.4028,
'subsample': 0.4278,
'min_child_samples': 25.65,
'min_child_weight': 0.6138,
'min_split_gain': 0.7354,
'num_leaves': 62.68,
'reg_alpha': 0.2889,
'reg_lambda': 7.875
"""

drop_features_vc1 = ['ZN_minus_ZN_YEAR_COMPONENT_median', 'ANONYMOUS_1_CU', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_COMPONENT_min', 'PQINDEX_divide_PQINDEX_YEAR_min', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_FE_woe_bin_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_min', 'V40_minus_V40_FE_woe_bin_sum', 'PQINDEX_minus_PQINDEX_FE_woe_bin_sum', 'V40_minus_V40_YEAR_COMPONENT_min', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_median', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_sum', 'FE_divide_FE_YEAR_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_COMPONENT_ARBITRARY_median', 'PQINDEX_minus_PQINDEX_YEAR_COMPONENT_max', 'PQINDEX_minus_PQINDEX_YEAR_COMPONENT_median', 'FE_minus_FE_YEAR_COMPONENT_median', 'ZN_divide_ZN_YEAR_min', 'PQINDEX_divide_PQINDEX_YEAR_median', 'FE_minus_FE_YEAR_COMPONENT_max', 'ZN_divide_ZN_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_FE_woe_bin_median', 'ANONYMOUS_1_ZN_YEAR_COMPONENT_max', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_COMPONENT_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_COMPONENT_sum', 'ANONYMOUS_1_FE_YEAR_median', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_FE_woe_bin_sum', 'CU_divide_CU_COMPONENT_ARBITRARY_sum', 'FE_divide_FE_YEAR_median', 'ANONYMOUS_2_YEAR_max', 'FE_divide_FE_FE_woe_bin_min', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_FE_woe_bin_sum', 'V40_minus_V40_YEAR_COMPONENT_max', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_sum', 'V40_divide_V40_FE_woe_bin_min', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_COMPONENT_median', 'ZN_minus_ZN_YEAR_COMPONENT_min', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_COMPONENT_max', 'V40_YEAR_median', 'V40_minus_V40_YEAR_sum', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_max', 'NI_V40', 'V40_minus_V40_COMPONENT_ARBITRARY_min', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_max', 'ZN_minus_ZN_YEAR_min', 'FE_divide_FE_YEAR_COMPONENT_max', 'ZN_divide_ZN_FE_woe_bin_min', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_FE_woe_bin_median', 'CU_minus_CU_YEAR_COMPONENT_median', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_max', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_FE_woe_bin_sum', 'ZN_divide_ZN_YEAR_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_FE_woe_bin_max', 'ANONYMOUS_2_divide_ANONYMOUS_2_FE_woe_bin_max', 'ZN_minus_ZN_FE_woe_bin_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_COMPONENT_ARBITRARY_max', 'FE_minus_FE_YEAR_max', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_COMPONENT_ARBITRARY_min', 'CU_YEAR_COMPONENT_median', 'cluster_no_by_ANONYMOUS_1_FE_median', 'ZN_YEAR_COMPONENT_sum', 'CU_minus_CU_FE_woe_bin_max', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_COMPONENT_median', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_median', 'CU_divide_CU_YEAR_COMPONENT_max', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_FE_woe_bin_max', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_YEAR_COMPONENT_min', 'CU_divide_CU_YEAR_max', 'ZN_divide_ZN_COMPONENT_ARBITRARY_max', 'FE_divide_FE_FE_woe_bin_max', 'cluster_no_by_ANONYMOUS_1_median', 'V40_divide_V40_FE_woe_bin_sum', 'V40_divide_V40_COMPONENT_ARBITRARY_max', 'ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_COMPONENT_sum', 'FE_YEAR_COMPONENT_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_COMPONENT_ARBITRARY_median', 'FE_divide_FE_COMPONENT_ARBITRARY_median', 'V40_divide_V40_YEAR_COMPONENT_max', 'V40_minus_V40_YEAR_min', 'ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_median', 'FE_minus_FE_FE_woe_bin_median', 'FE_minus_FE_YEAR_median', 'ANONYMOUS_1_FE_YEAR_COMPONENT_sum', 'ZN_minus_ZN_FE_woe_bin_sum', 'CU_minus_CU_FE_woe_bin_sum', 'V40', 'ANONYMOUS_1_ZN_woe_bin', 'FE_divide_FE_COMPONENT_ARBITRARY_min', 'CU_divide_CU_FE_woe_bin_sum', 'cluster_no_by_ANONYMOUS_2_mean', 'CU', 'V40_minus_V40_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_minus_ANONYMOUS_1_FE_woe_bin_median', 'V40_divide_V40_YEAR_min', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_COMPONENT_ARBITRARY_sum', 'ZN_minus_ZN_YEAR_COMPONENT_max', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_median', 'cluster_no_by_ANONYMOUS_1_V40_mean', 'FE_minus_FE_FE_woe_bin_min', 'PQINDEX_divide_PQINDEX_YEAR_max', 'ANONYMOUS_2_minus_ANONYMOUS_2_FE_woe_bin_sum', 'FE_divide_FE_YEAR_sum', 'V40_minus_V40_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_FE_woe_bin_max', 'PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_sum', 'ANONYMOUS_1_V40', 'ZN_divide_ZN_YEAR_sum', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_COMPONENT_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_FE_woe_bin_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_COMPONENT_ARBITRARY_sum', 'PQINDEX_minus_PQINDEX_FE_woe_bin_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_max', 'PQINDEX_divide_PQINDEX_FE_woe_bin_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_sum', 'ZN_divide_ZN_YEAR_median', 'ZN_minus_ZN_YEAR_COMPONENT_sum', 'ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_COMPONENT_median', 'V40_minus_V40_YEAR_max', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_FE_woe_bin_min', 'CR_FE', 'FE_V40', 'FE_divide_FE_YEAR_COMPONENT_min', 'V40_divide_V40_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE', 'FE_divide_FE_COMPONENT_ARBITRARY_max', 'PQINDEX_minus_PQINDEX_YEAR_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_median', 'ZN_divide_ZN_YEAR_COMPONENT_median', 'ANONYMOUS_1_minus_ANONYMOUS_1_COMPONENT_ARBITRARY_max', 'CU_divide_CU_YEAR_median', 'ZN_divide_ZN_YEAR_COMPONENT_min', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_FE_woe_bin_max', 'PQINDEX_divide_PQINDEX_FE_woe_bin_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_COMPONENT_max', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_median', 'ANONYMOUS_1_MO', 'CU_FE', 'ZN_minus_ZN_YEAR_sum', 'ZN_minus_ZN_COMPONENT_ARBITRARY_max', 'V40_divide_V40_FE_woe_bin_max', 'PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_FE_woe_bin_sum', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_median', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_YEAR_min', 'PQINDEX_divide_PQINDEX_FE_woe_bin_min', 'PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_min', 'ANONYMOUS_2_divide_ANONYMOUS_2_COMPONENT_ARBITRARY_median', 'PQINDEX_YEAR_COMPONENT_sum', 'PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_median', 'PQINDEX_minus_PQINDEX_YEAR_COMPONENT_sum', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_max', 'CU_ZN', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_COMPONENT_max', 'ZN_divide_ZN_YEAR_COMPONENT_max', 'V40_minus_V40_FE_woe_bin_min', 'NI_ZN', 'V40_divide_V40_FE_woe_bin_median', 'ZN_minus_ZN_YEAR_max', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_median', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_max', 'ANONYMOUS_1_ZN', 'PQINDEX_YEAR_median', 'NI', 'FE_ZN', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_max', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_FE_woe_bin_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_median', 'CR_V40', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_COMPONENT_median', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_min', 'ANONYMOUS_2_divide_ANONYMOUS_2_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_COMPONENT_ARBITRARY_min', 'MO_V40', 'PQINDEX_minus_PQINDEX_FE_woe_bin_max', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_min', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_median', 'FE_minus_FE_YEAR_COMPONENT_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_max', 'cluster_no_by_ANONYMOUS_1_V40_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_max', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_COMPONENT_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_min', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_FE_woe_bin_median', 'ANONYMOUS_2_MO', 'PQINDEX_log1', 'CU_minus_CU_YEAR_COMPONENT_max', 'FE_NI', 'ANONYMOUS_1_divide_ANONYMOUS_1_FE_woe_bin_max', 'CU_divide_CU_FE_woe_bin_median']
drop_features_vc2 = ['PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_median', 'ZN_minus_ZN_YEAR_COMPONENT_min', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_median', 'PQINDEX_divide_PQINDEX_YEAR_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_max', 'FE_divide_FE_FE_woe_bin_min', 'V40_divide_V40_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_FE_woe_bin_sum', 'FE_divide_FE_YEAR_COMPONENT_sum', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_median', 'FE_divide_FE_COMPONENT_ARBITRARY_max', 'CU_divide_CU_YEAR_max', 'FE_divide_FE_YEAR_sum', 'V40_divide_V40_YEAR_min', 'FE_divide_FE_FE_woe_bin_max', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_COMPONENT_sum', 'ANONYMOUS_2_divide_ANONYMOUS_2_FE_woe_bin_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_COMPONENT_sum', 'V40_divide_V40_FE_woe_bin_sum', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE_YEAR_median', 'ZN_divide_ZN_YEAR_min', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_min', 'ZN_minus_ZN_FE_woe_bin_sum', 'PQINDEX_divide_PQINDEX_YEAR_min', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_FE_woe_bin_min', 'CU_divide_CU_YEAR_COMPONENT_min', 'ANONYMOUS_2_YEAR_max', 'CU_minus_CU_FE_woe_bin_sum', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_max', 'FE_minus_FE_FE_woe_bin_sum', 'FE_divide_FE_YEAR_max', 'PQINDEX_minus_PQINDEX_FE_woe_bin_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_sum', 'FE_minus_FE_YEAR_max', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_COMPONENT_min', 'FE_minus_FE_FE_woe_bin_min', 'cluster_no_by_ANONYMOUS_1_FE_median', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_COMPONENT_median', 'ZN_minus_ZN_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_FE_woe_bin_sum', 'ZN_minus_ZN_YEAR_sum', 'NI_V40', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_max', 'CU_minus_CU_YEAR_sum', 'V40_minus_V40_FE_woe_bin_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_median', 'PQINDEX_divide_PQINDEX_FE_woe_bin_max', 'ZN_minus_ZN_FE_woe_bin_max', 'V40_minus_V40_COMPONENT_ARBITRARY_min', 'FE_minus_FE_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE', 'PQINDEX_minus_PQINDEX_FE_woe_bin_median', 'PQINDEX_minus_PQINDEX_YEAR_max', 'FE_YEAR_COMPONENT_median', 'V40_divide_V40_COMPONENT_ARBITRARY_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_sum', 'ZN_minus_ZN_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_CR', 'CU_minus_CU_YEAR_COMPONENT_max', 'CU_minus_CU_FE_woe_bin_max', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_COMPONENT_max', 'FE', 'V40_minus_V40_YEAR_min', 'ZN_divide_ZN_YEAR_COMPONENT_max', 'V40_divide_V40_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_FE_woe_bin_min', 'V40_divide_V40_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_max', 'CU_divide_CU_YEAR_COMPONENT_max', 'NI_ZN', 'ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_COMPONENT_sum', 'FE_YEAR_COMPONENT_max', 'FE_divide_FE_YEAR_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_FE_woe_bin_median', 'ZN_divide_ZN_YEAR_COMPONENT_min', 'CU_divide_CU_YEAR_COMPONENT_sum', 'FE_minus_FE_FE_woe_bin_median', 'ANONYMOUS_2_divide_ANONYMOUS_2_FE_woe_bin_sum', 'CU_divide_CU_FE_woe_bin_sum', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_COMPONENT_ARBITRARY_median', 'FE_divide_FE_YEAR_COMPONENT_min', 'V40_minus_V40_FE_woe_bin_sum', 'ANONYMOUS_2_divide_ANONYMOUS_2_COMPONENT_ARBITRARY_max', 'V40_divide_V40_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_median', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_FE_woe_bin_sum', 'PQINDEX_minus_PQINDEX_FE_woe_bin_sum', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_COMPONENT_max', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_sum', 'CR_ZN', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_max', 'CR_V40', 'V40_minus_V40_FE_woe_bin_max', 'V40_minus_V40_YEAR_COMPONENT_sum', 'ZN_divide_ZN_COMPONENT_ARBITRARY_max', 'CU_divide_CU_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_FE_woe_bin_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_min', 'ZN_minus_ZN_YEAR_COMPONENT_max', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_median', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_COMPONENT_sum', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_min', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_median', 'FE_minus_FE_YEAR_median', 'ANONYMOUS_1_minus_ANONYMOUS_1_COMPONENT_ARBITRARY_min', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_max', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_COMPONENT_ARBITRARY_max', 'V40_divide_V40_YEAR_max', 'ANONYMOUS_2_MO', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_YEAR_COMPONENT_sum', 'V40_divide_V40_YEAR_COMPONENT_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_COMPONENT_ARBITRARY_min', 'PQINDEX_divide_PQINDEX_FE_woe_bin_sum', 'ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_COMPONENT_median', 'MO_NI', 'MO_ZN', 'ANONYMOUS_1_minus_ANONYMOUS_1_FE_woe_bin_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_median', 'ZN_minus_ZN_YEAR_COMPONENT_median', 'CR_MO', 'CR_FE', 'ZN_divide_ZN_FE_woe_bin_max', 'V40_minus_V40_YEAR_COMPONENT_max', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_median', 'FE_divide_FE_YEAR_min', 'PQINDEX_minus_PQINDEX_YEAR_COMPONENT_median', 'V40_woe_bin', 'PQINDEX_divide_PQINDEX_FE_woe_bin_min', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_median', 'CU_YEAR_COMPONENT_max', 'ANONYMOUS_2_YEAR_COMPONENT_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_FE_woe_bin_min', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_FE_woe_bin_max', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_FE_woe_bin_median', 'ZN_minus_ZN_FE_woe_bin_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_sum', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_min', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_sum', 'FE_NI', 'ANONYMOUS_1_divide_ANONYMOUS_1_FE_woe_bin_sum', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_YEAR_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_COMPONENT_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_sum', 'NI', 'ZN_minus_ZN_YEAR_COMPONENT_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_FE_woe_bin_max', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_max', 'ZN_divide_ZN_YEAR_COMPONENT_median', 'V40_YEAR_median', 'PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_min', 'V40_divide_V40_FE_woe_bin_median', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_COMPONENT_max', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_FE_woe_bin_min', 'ANONYMOUS_1_CU', 'CU_divide_CU_FE_woe_bin_max', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_FE_woe_bin_max', 'V40', 'ANONYMOUS_1_divide_ANONYMOUS_1_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_log1', 'ANONYMOUS_2_minus_ANONYMOUS_2_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_max', 'V40_divide_V40_FE_woe_bin_min', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_FE_woe_bin_sum']
drop_features_vc3 = ['PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_median', 'ZN_minus_ZN_YEAR_COMPONENT_min', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_median', 'PQINDEX_divide_PQINDEX_YEAR_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_max', 'FE_divide_FE_FE_woe_bin_min', 'V40_divide_V40_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_FE_woe_bin_sum', 'FE_divide_FE_YEAR_COMPONENT_sum', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_median', 'FE_divide_FE_COMPONENT_ARBITRARY_max', 'CU_divide_CU_YEAR_max', 'FE_divide_FE_YEAR_sum', 'V40_divide_V40_YEAR_min', 'FE_divide_FE_FE_woe_bin_max', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_COMPONENT_sum', 'ANONYMOUS_2_divide_ANONYMOUS_2_FE_woe_bin_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_COMPONENT_sum', 'V40_divide_V40_FE_woe_bin_sum', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE_YEAR_median', 'ZN_divide_ZN_YEAR_min', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_min', 'ZN_minus_ZN_FE_woe_bin_sum', 'PQINDEX_divide_PQINDEX_YEAR_min', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_FE_woe_bin_min', 'CU_divide_CU_YEAR_COMPONENT_min', 'ANONYMOUS_2_YEAR_max', 'CU_minus_CU_FE_woe_bin_sum', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_max', 'FE_minus_FE_FE_woe_bin_sum', 'FE_divide_FE_YEAR_max', 'PQINDEX_minus_PQINDEX_FE_woe_bin_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_sum', 'FE_minus_FE_YEAR_max', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_COMPONENT_min', 'FE_minus_FE_FE_woe_bin_min', 'cluster_no_by_ANONYMOUS_1_FE_median', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_COMPONENT_median', 'ZN_minus_ZN_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_FE_woe_bin_sum', 'ZN_minus_ZN_YEAR_sum', 'NI_V40', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_max', 'CU_minus_CU_YEAR_sum', 'V40_minus_V40_FE_woe_bin_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_median', 'PQINDEX_divide_PQINDEX_FE_woe_bin_max', 'ZN_minus_ZN_FE_woe_bin_max', 'V40_minus_V40_COMPONENT_ARBITRARY_min', 'FE_minus_FE_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE', 'PQINDEX_minus_PQINDEX_FE_woe_bin_median', 'PQINDEX_minus_PQINDEX_YEAR_max', 'FE_YEAR_COMPONENT_median', 'V40_divide_V40_COMPONENT_ARBITRARY_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_sum', 'ZN_minus_ZN_COMPONENT_ARBITRARY_max']
drop_features_vc4 = ['PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_median', 'ZN_minus_ZN_YEAR_COMPONENT_min', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_median', 'PQINDEX_divide_PQINDEX_YEAR_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_max', 'FE_divide_FE_FE_woe_bin_min', 'V40_divide_V40_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_max']

opt_params = {
    'learning_rate': (0.02, 0.08),
    'num_leaves': (24, 80),
    'colsample_bytree': (0.2, 0.8),
    'subsample': (0.2, 0.8),
    'max_depth': (3, 8),
    'reg_alpha': (0.01, 10),
    'reg_lambda': (1, 10),
    'min_split_gain': (0.01, 1),
    'min_child_weight': (0.001,1), # (0.001,50) <- 이 부분 제거하고 튜닝해보기 ...   
    'min_child_samples':(10, 50)
}
    
# tune
opt_params = bayes_parameter_opt_lgb(train, opt_params, drop_features_vc1, init_round=100, opt_round=100, n_folds=5, random_seed=176, n_estimators=10000)

# check 
print(opt_params[1])

# re-train ml with the best one
train_model(train,test,opt_params[1],True,5,drop_features_vc1,seed_num=176)

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | min_ch... | min_sp... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.69     |  0.4145   |  0.03276  |  4.676    |  41.47    |  0.8811   |  0.8651   |  46.63    |  6.17     |  4.265    |  0.3182   |
|  2        |  0.7017   |  0.3335   |  0.07927  |  4.733    |  21.94    |  0.3311   |  0.1729   |  77.63    |  2.723    |  9.365    |  0.7622   |
|  3        |  0.6989   |  0.3783   |  0.04372  |  3.792    |  31.63    |  0.3362   |  0.8245   |  26.51    |  6.938    |  1.938    |  0.7214   |
|  4        |  0.6888   |  0.783    |  0.05741  |  5.922    |  36.85    |  0.377    |  0.3675   |  49.05    |  8.657    |  6.6      |  0.6131   |
|  5        |  0.7002   |  0.3783   |  0.06092  |  3.57     |  29.36    |  0.5373   |  0.5422   |  49.93    |  6.243    |  7

feature
ANONYMOUS_1_divide_ANONYMOUS_1_COMPONENT_ARBITRARY_median          160
ANONYMOUS_1                                                        141
FE_divide_FE_FE_woe_bin_median                                     136
CU_divide_CU_YEAR_sum                                              105
ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_min                            102
ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_max                       99
V40_divide_V40_YEAR_COMPONENT_min                                   99
FE_divide_FE_YEAR_COMPONENT_sum                                     95
ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_YEAR_COMPONENT_max           93
ANONYMOUS_1_log1                                                    92
ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_YEAR_max                     92
ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_COMPONENT_max             91
ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_YEAR_sum                     88
ANONYMOUS_2                                                         8

thred: 0.1659
ncol 310
auc: 0.7037415059414243
f1: 0.5895183191182717
Target ratio(real): 0.08534941468605889
Target ratio(pred): 0.0759807978811455
Target sum: 459
CPU times: user 2h 39min 14s, sys: 1min 35s, total: 2h 40min 50s
Wall time: 41min 58s
